In [ ]:
from pathlib import Path
import sys

lib_path = str(Path('../').resolve())
sys.path.insert(0, lib_path)
lib_path

In [ ]:
from crops import CropModels
from utils.weather import get_weather_df


csv_fp = Path('../../../assets') / 'sample.csv'

weather_df = get_weather_df(csv_fp)
crop_models = {ModelClass.key: ModelClass for ModelClass in CropModels}


### 고추 - 이상 화분
> 개화전 13~17일 평균기온 30℃ 이상에서 이상화분 발생

In [ ]:
crop_model = crop_models.get('chili')()

crop_model.set_weather_data(weather_df)
df = crop_model.gdd_weather_df.copy()

period = 10  # 10일 마다 고온장해 확률값을 나타내는 대표값을 계산

df['pollen_damaged'] = 0
df.loc[df['tavg'] >= 30, 'pollen_damaged'] = 1  # 평균기온 30도 이상에서 이상화분
prob_df = df.groupby((df.index - 1) // period + 1).agg({'pollen_damaged': 'mean'})  # 10일 동안 이상화분 확률값 계산
display(prob_df['pollen_damaged'].describe())

data = []
for k, v in prob_df['pollen_damaged'].to_dict().items():
    data.append({
        'doy_range': (k*period - period + 1, min(k*period, 366)),
        'prob': v
    })
    
data[10:20]

### 옥수수 - 임실률 감소
> 고온 장해: 출웅기(출사전 6~7일) 최고기온 35℃ 초과 임실률 감소

In [ ]:
crop_model = crop_models.get('corn')()

crop_model.set_weather_data(weather_df)
df = crop_model.gdd_weather_df.copy()

period = 10  # 10일 마다 고온장해 확률값을 나타내는 대표값을 계산

df['ripening_reduced'] = 0
df.loc[df['tmax'] > 35, 'ripening_reduced'] = 1  # 최고기온 35도 초과에서 임실률 감소
prob_df = df.groupby((df.index - 1) // period + 1).agg({'ripening_reduced': 'mean'})  # 10일 동안 임실률감소 확률값 계산
display(prob_df['ripening_reduced'].describe())

data = []
for k, v in prob_df['ripening_reduced'].to_dict().items():
    data.append({
        'doy_range': (k*period - period + 1, min(k*period, 366)),
        'prob': v
    })
    
data[10:20]

### 벼 - 안전출수기
> 출수기 이후 40일 평균기온 21~24도 사이 22.5도가 최적 출수기
<br>
> 범위에서 벗어나 1도 차이마다 10%씩 수량감소


In [ ]:
# 안전출수기
crop_model = crop_models.get('middleLateMatureRice')()

crop_model.set_weather_data(weather_df)
df = crop_model.gdd_weather_df.copy()
start_doy = crop_model.start_doy
if hasattr(crop_model, 'heading_gdd_range'):
    heading_range = [
        max(start_doy, crop_model.get_event_end_doy(start_doy, gdd))
        for gdd in crop_model.heading_gdd_range
    ]
elif hasattr(crop_model, 'heading_gdd'):
    heading = crop_model.get_event_end_doy(start_doy, crop_model.heading_gdd)
    heading_range = [
        max(start_doy, heading - 5), heading + 9
    ]
else:
    raise AttributeError(f'CropModel {crop_model.key} is incomplete')

window_size = 40  # 40 일

df['40days_tavg'] = df.rolling(window_size + 1).agg({'tavg': 'mean'}).shift(-window_size)['tavg']  # 40일 동안의 평균기온
harvest_decrease_by_cold = (
    df.loc[heading_range[0]: heading_range[1], '40days_tavg']
    - 21).clip(upper=0).round(0)
harvest_decrease_by_heat = (
    24
    - df.loc[heading_range[0]: heading_range[1], '40days_tavg']
    ).clip(upper=0).round(0)

n_harvest_decrease = (
    (harvest_decrease_by_cold < 0).sum() +
    (harvest_decrease_by_heat < 0).sum()
)

n_harvest_decrease

### 감자 - 비대정지
> 고온피해: 덩이줄기 비대기(정식 30일 후 ~ 수확 10일전) 최고기온 27~30℃ 덩이줄기 비대 정지


In [ ]:
crop_model = crop_models.get('potato')()

crop_model.set_weather_data(weather_df)
df = crop_model.gdd_weather_df.copy()

period = 10  # 10일 마다 고온장해 확률값을 나타내는 대표값을 계산

df['growth_stopped'] = 0
df.loc[
  (df['tmax'] >= 27) & (df['tmax'] <= 30), 
  'growth_stopped'
] = 1  # 최고기온 27~30도에서 감자 비대정지
prob_df = df.groupby((df.index - 1) // period + 1).agg({'growth_stopped': 'mean'})  # 10일 동안 비대정지 확률값 계산
display(prob_df['growth_stopped'].describe())

data = []
for k, v in prob_df['growth_stopped'].to_dict().items():
    data.append({
        'doy_range': (k*period - period + 1, min(k*period, 366)),
        'prob': v
    })
    
data[10:20]

### 마늘 - 생육정지
> 고온 피해: 쪽 분화 ~ 수확전 25℃ 이상에서 생육정지
<br>
> 쪽분화 1월21일~3월 21일

In [ ]:
crop_model = crop_models.get('garlic')()

crop_model.set_weather_data(weather_df)
df = crop_model.gdd_weather_df.copy()

period = 10  # 10일 마다 고온장해 확률값을 나타내는 대표값을 계산

df['develop_stopped'] = 0
df.loc[df['tmax'] >= 25, 'develop_stopped'] = 1  # 최고기온 25도 이상에서 마늘 생육정지
prob_df = df.groupby((df.index - 1) // period + 1).agg({'develop_stopped': 'mean'})  # 10일 동안 비대정지 확률값 계산
display(prob_df['develop_stopped'].describe())

data = []
for k, v in prob_df['develop_stopped'].to_dict().items():
    data.append({
        'doy_range': (k*period - period + 1, min(k*period, 366)),
        'prob': v
    })
    
data[10:20]

### 양파 - 생육둔화
> 고온 피해: 구비대기 (수확40일전) 25℃ 이상에서 생육둔화

In [ ]:
crop_model = crop_models.get('onion')()

crop_model.set_weather_data(weather_df)
df = crop_model.gdd_weather_df.copy()

period = 10  # 10일 마다 고온장해 확률값을 나타내는 대표값을 계산

df['develop_slowed'] = 0
df.loc[df['tmax'] >= 25, 'develop_slowed'] = 1  # 최고기온 25도 이상에서 양파 생육정지
prob_df = df.groupby((df.index - 1) // period + 1).agg({'develop_slowed': 'mean'})  # 10일 동안 비대정지 확률값 계산
display(prob_df['develop_slowed'].describe())

data = []
for k, v in prob_df['develop_slowed'].to_dict().items():
    data.append({
        'doy_range': (k*period - period + 1, min(k*period, 366)),
        'prob': v
    })
    
data[10:20]

### 참깨 - 조기 낙화
> 고온피해: 개화기 40℃ 이상 고온 조기낙화 등숙률 저하

In [ ]:
crop_model = crop_models.get('sesame')()

crop_model.set_weather_data(weather_df)
df = crop_model.gdd_weather_df.copy()

period = 10  # 10일 마다 고온장해 확률값을 나타내는 대표값을 계산

df['premature_abscission'] = 0
df.loc[df['tmax'] >= 40, 'premature_abscission'] = 1  # 최고기온 40도 이상에서 참깨 조기낙화
prob_df = df.groupby((df.index - 1) // period + 1).agg({'premature_abscission': 'mean'})  # 10일 동안 비대정지 확률값 계산
display(prob_df['premature_abscission'].describe())

data = []
for k, v in prob_df['premature_abscission'].to_dict().items():
    data.append({
        'doy_range': (k*period - period + 1, min(k*period, 366)),
        'prob': v
    })
    
data[10:20]